# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [22]:
!pip install geopandas

In [23]:
# Dependencies and Setup
import hvplot.pandas
import geopandas as gpd
import pandas as pd
import requests
import matplotlib.pyplot as plt
import holoviews as hv


# Import API key
from api_keys import geoapify_key

In [24]:
!pip install --upgrade hvplot holoviews

In [25]:
!pip install geoviews

In [26]:
!pip install cartopy


In [27]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,adamstown,-25.0660,-130.1015,76.86,67.0,43.0,9.08,PN,1.710011e+09
1,1,agadez,19.7500,10.2500,81.37,9.0,0.0,11.25,NE,1.710011e+09
2,2,taiohae,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,bilibino,68.0546,166.4372,-2.52,99.0,100.0,2.51,RU,1.710011e+09
4,4,port-aux-francais,-49.3500,70.2167,45.52,97.0,100.0,31.16,TF,1.710011e+09


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [34]:
#!pip install folium

In [46]:
##TO DO MY CODE HERE##


# Create a scatter plot with hvplot (code sourced from https://holoviz.org/tutorial/Composing_Plots.html, 1/27/24)

scatter_plot = city_data_df.hvplot.points(
    x='Lng', y='Lat', c='Humidity', hover_cols=['City', 'Humidity'], title='City Humidity Map', line_color='black')


# # Display the plot
scatter_plot


:Points   [Lng,Lat]   (Humidity,City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [48]:
##TO DO MY CODE HERE##

# Narrow down cities that fit criteria and drop any results with null values
city_data_locations = city_data_df[(city_data_df['Max Temp'] > 65) & (city_data_df['Humidity'] < 65) & (city_data_df['Wind Speed'] < 15) 
            & (city_data_df['Cloudiness'] < 45)]

##TO DO MY CODE HERE##
# Drop any rows with null values
city_data_locations.dropna(axis=0)

##TO DO MY CODE HERE##
# Display sample data
city_data_locations.count()

City_ID       66
City          66
Lat           66
Lng           66
Max Temp      66
Humidity      66
Cloudiness    66
Wind Speed    66
Country       66
Date          66
dtype: int64

### Step 3: Create a new DataFrame called `hotel_df`.

In [49]:
##TO DO MY CODE HERE##
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_locations.copy()

##TO DO MY CODE HERE##
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ('')

##TO DO MY CODE HERE##
# Display sample data
hotel_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
1,1,agadez,19.7500,10.2500,81.37,9.0,0.0,11.25,NE,1.710011e+09,
19,19,awjilah,29.1081,21.2869,66.92,34.0,0.0,6.60,LY,1.710011e+09,
54,54,biltine,14.5333,20.9167,88.88,11.0,0.0,6.44,TD,1.710011e+09,
63,63,san jorge,-31.8962,-61.8598,89.83,51.0,44.0,5.66,AR,1.710011e+09,
76,76,petrolina,-9.3986,-40.5008,89.65,51.0,40.0,6.91,BR,1.710011e+09,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [50]:

##TO DO MY CODE HERE##
# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    ##TO DO MY CODE HERE##
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']
    
    
##TO DO MY CODE HERE##
# Set parameters to search for a hotel TO DO MY CODE HERE
#ChatGPT 2024, personal communication, January 27,2024
    radius = 10000
    filters = f"circle:{longitude},{latitude},{radius}"
    bias = f"proximity:{longitude},{latitude}"
    params = {
    "apiKey":geoapify_key,
    "filters":filters,
    "bias":bias,
    "type":"accommodation.hotel"
        
    }
    
##TO DO MY CODE HERE##
# Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    #ChatGPT 2024, personal communication, January 27,2024 TO DO MY CODE HERE
   
    params["filters"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
      
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


##TO DO MY CODE HERE##
# Make an API request using the params dictionary
    name_address = requests.get(base_url, params=params)
      
        
##TO DO MY CODE HERE##
# Convert the API response to JSON format
    name_address = name_address.json()
    
    #print to debug
    #print("API Response:", name_address) #  -- this is no good I get this output
   # API Response: {'statusCode': 400, 'error': 'Bad Request', 'message': '"filter[0]" does not match any of the allowed types'}
    #tiksi - nearest hotel: No hotel found
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head()

Starting hotel search
agadez - nearest hotel: Dabaga Village
awjilah - nearest hotel: No hotel found
biltine - nearest hotel: فندق ثلاث نجوم بلتن
san jorge - nearest hotel: Hotel Monumental
petrolina - nearest hotel: Neuman
boke - nearest hotel: Hôtel Rio Nunez
port lincoln - nearest hotel: Boston Hotel
port augusta - nearest hotel: Hotel Commonwealth
talara - nearest hotel: Hospedaje El Aypate
kingston - nearest hotel: Aphrodite Hotel
al wakrah - nearest hotel: Sealine Beach Resort
pisco - nearest hotel: La Portada
ad dilam - nearest hotel: شقق فندقية
cabo san lucas - nearest hotel: Comfort Rooms
moree - nearest hotel: Royal Hotel
lionel town - nearest hotel: Milk River Mineral Bath and Spa
phalaborwa - nearest hotel: Hans Merensky Hotel & Estate
frankfort - nearest hotel: No hotel found
chakradharpur - nearest hotel: Ginger
port sudan - nearest hotel: Ranya Hotel
taoudenni - nearest hotel: فندق السايح
ciudad lazaro cardenas - nearest hotel: Hotel Sol del Pacífico
goundi - nearest hot

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
1,1,agadez,19.7500,10.2500,81.37,9.0,0.0,11.25,NE,1.710011e+09,Dabaga Village
19,19,awjilah,29.1081,21.2869,66.92,34.0,0.0,6.60,LY,1.710011e+09,No hotel found
54,54,biltine,14.5333,20.9167,88.88,11.0,0.0,6.44,TD,1.710011e+09,فندق ثلاث نجوم بلتن
63,63,san jorge,-31.8962,-61.8598,89.83,51.0,44.0,5.66,AR,1.710011e+09,Hotel Monumental
76,76,petrolina,-9.3986,-40.5008,89.65,51.0,40.0,6.91,BR,1.710011e+09,Neuman


In [51]:
hotel_df.count()

City_ID       66
City          66
Lat           66
Lng           66
Max Temp      66
Humidity      66
Cloudiness    66
Wind Speed    66
Country       66
Date          66
Hotel Name    66
dtype: int64

In [52]:
hotel_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
1,1,agadez,19.7500,10.2500,81.37,9.0,0.0,11.25,NE,1.710011e+09,Dabaga Village
19,19,awjilah,29.1081,21.2869,66.92,34.0,0.0,6.60,LY,1.710011e+09,No hotel found
54,54,biltine,14.5333,20.9167,88.88,11.0,0.0,6.44,TD,1.710011e+09,فندق ثلاث نجوم بلتن
63,63,san jorge,-31.8962,-61.8598,89.83,51.0,44.0,5.66,AR,1.710011e+09,Hotel Monumental
76,76,petrolina,-9.3986,-40.5008,89.65,51.0,40.0,6.91,BR,1.710011e+09,Neuman


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [53]:
%%capture --no-display
##TO DO MY CODE HERE##

# Configure the map plot

scatter_plot_2 = hotel_df.hvplot.points(
    x='Lng', y='Lat', c='Humidity', hover_cols=['City', 'Humidity', 'Country', 'Hotel Name'], title='City Humidity Map', line_color='black')


##TO DO MY CODE HERE##
# Display the map
scatter_plot_2

:Points   [Lng,Lat]   (Humidity,City,Country,Hotel Name)